In [1]:
import json
from tqdm import tqdm

with open("structured_bios_new_english_fixed.json", "r", encoding="utf-8") as f:
    people = json.load(f)

In [2]:
def years_overlap(y1, y2):
    """iki yıl aralığı çakışıyor mu (örn: 2005-2010 vs 2008-2012)"""
    try:
        s1, e1 = [int(y) for y in y1.split("-")]
        s2, e2 = [int(y) for y in y2.split("-")]
        return max(s1, s2) <= min(e1, e2)
    except:
        return False


In [3]:
results = []

for i in tqdm(range(len(people))):
    for j in range(i + 1, len(people)):
        p1 = people[i]
        p2 = people[j]
        rels = []

        # AYNI_MEMLEKETTEN
        if p1["dogum_yeri"] == p2["dogum_yeri"]:
            rels.append("AYNI_MEMLEKETTEN")

        # OKUL ilişkileri (ve yıl çakışması)
        for okul, yil in [("ilkokul", "ilkokul_yillari"), ("lise", "lise_yillari"),
                          ("universite", "universite_yillari"), ("yuksek_lisans", "yuksek_lisans_yillari"),
                          ("doktora", "doktora_yillari")]:
            if p1.get(okul) and p1.get(okul) == p2.get(okul):
                if p1.get(yil) and p2.get(yil) and years_overlap(p1[yil], p2[yil]):
                    rel_ad = okul.upper() + "_ARKADASI"
                    rels.append(rel_ad)

        # MESLEKTAŞ
        if p1.get("bolum") and p1.get("bolum") == p2.get("bolum"):
            rels.append("MESLEKTAS")

        # AYNI_KURUMDA_CALISMIŞ
        ortak_kurum = set(p1.get("calistigi_kurumlar", [])) & set(p2.get("calistigi_kurumlar", []))
        if ortak_kurum:
            rels.append("AYNI_KURUMDA_CALISMIS")

        # ILGI_ALANI_ORTAKLIĞI
        if set(p1.get("hobiler", [])) & set(p2.get("hobiler", [])):
            rels.append("ILGI_ALANI_ORTAKLIGI")

        if rels:
            results.append({
                "source": p1["ad"],
                "target": p2["ad"],
                "relations": rels
            })


100%|██████████| 243/243 [00:01<00:00, 239.59it/s]


In [4]:
# Sonuçları kaydet
with open("automated_relationships.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"{len(results)} ilişki bulundu.")

13867 ilişki bulundu.
